In [25]:
import pandas as pd
import itertools as it

from pandas import DataFrame

from scipy import stats
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns

from copy import deepcopy
import numpy as np
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = (16, 9)
plt.style.use('ggplot')

In [2]:
stars_filepath = 'review_stars_rest_subset.txt'
review_txt_filepath = 'review_text_rest_subset.txt'
business_filepath = 'review_business_rest_subset.txt'
user_filepath = 'review_user_rest_subset.txt'

In [3]:
with open(business_filepath) as f:
    business = f.readlines()
# remove whitespace characters like `\n` at the end of each line
business = [x.strip() for x in business]

In [4]:
bus_set = frozenset(business)
print(len(bus_set))

52810


In [5]:
lda_file = 'output_file_test_minimal_codecs_substr.txt'

In [100]:
lda_read = pd.io.parsers.read_csv(lda_file,sep=",", header=None)
len(lda_read)

13728

In [101]:
business_id = list(bus_set)[0:len(lda_read)]
len(business_id)

13728

In [102]:
lda_read.head()

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,0.0000,0.0000,0.0000,0.0623,0.0,0.0,0.0000,0.0000,0.0,0.0000,...,0.0000,0.000,0.0,0.0171,0.0276,0.0000,0.0593,0.0886,0.0,0.1217
1,0.2733,0.1195,0.1067,0.0000,0.0,0.0,0.0000,0.0000,0.0,0.0461,...,0.0000,0.044,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.0000
2,0.0000,0.0221,0.1126,0.0000,0.0,0.0,0.0000,0.0000,0.0,0.0190,...,0.0000,0.000,0.0,0.0000,0.0214,0.0000,0.0000,0.0279,0.0,0.0601
3,0.0000,0.0140,0.0000,0.0000,0.0,0.0,0.0412,0.0468,0.0,0.0116,...,0.0000,0.000,0.0,0.0000,0.0281,0.1107,0.0000,0.0156,0.0,0.0000
4,0.0000,0.1261,0.0000,0.0000,0.0,0.0,0.0000,0.0000,0.0,0.0668,...,0.2102,0.000,0.0,0.0000,0.0138,0.0000,0.0165,0.0000,0.0,0.1548


In [103]:
lda_read.insert(0, 'business_id',business_id)

In [104]:
lda_read.head()

,business_id,0,1,2,3,4,5,6,7,8,...,40,41,42,43,44,45,46,47,48,49
0,DZWAS8k2F18CrAojI8meuQ,0.0000,0.0000,0.0000,0.0623,0.0,0.0,0.0000,0.0000,0.0,...,0.0000,0.000,0.0,0.0171,0.0276,0.0000,0.0593,0.0886,0.0,0.1217
1,0i95sgY7pzYW9k88SOVJ8A,0.2733,0.1195,0.1067,0.0000,0.0,0.0,0.0000,0.0000,0.0,...,0.0000,0.044,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.0000
2,MjBp8R5yOPdcQ_-5mpNIpg,0.0000,0.0221,0.1126,0.0000,0.0,0.0,0.0000,0.0000,0.0,...,0.0000,0.000,0.0,0.0000,0.0214,0.0000,0.0000,0.0279,0.0,0.0601
3,nGqj3Hjx_daTMzFke7g6aQ,0.0000,0.0140,0.0000,0.0000,0.0,0.0,0.0412,0.0468,0.0,...,0.0000,0.000,0.0,0.0000,0.0281,0.1107,0.0000,0.0156,0.0,0.0000
4,QfD_KzfBGRZ1OLu1Vs1rrA,0.0000,0.1261,0.0000,0.0000,0.0,0.0,0.0000,0.0000,0.0,...,0.2102,0.000,0.0,0.0000,0.0138,0.0000,0.0165,0.0000,0.0,0.1548


In [105]:
topic_names = {0: u'chinese',
               1: u'thai',
               2: u'healthy',
               3: u'smell',
               4: u'japanese',
               5: u'toronto',
               6: u'service',
               7: u'experience',
               8: u'asian soup',
               9: u'grocery',
               10: u'parking',
               11: u'bar ambiance',
               12: u'uk',
               13: u'good service',
               14: u'fun ambiance',
               15: u'young',
               16: u'comfort food',
               17: u'greek',
               18: u'high end',
               19: u'hotwing',
               20: u'breakfast',
               21: u'sweet',
               22: u'wine & dine',
               23: u'pubs',
               24: u'good taste',
               25: u'na drinks',
               26: u'desserts',
               27: u'coffee shop',
               28: u'mexican',
               29: u'reviews',
               30: u'new york',
               31: u'general restaurant',
               32: u'beach',
               33: u'location',
               34: u'happy hour',
               35: u'amazing',
               36: u'vietnamese',
               37: u'time',
               38: u'vas legas',
               39: u'montreal',
               40: u'deli',
               41: u'buffet',
               42: u'bbq',
               43: u'french',
               44: u'money',
               45: u'street taco',
               46: u'pizza',
               47: u'airport & delivery',
               48: u'burger & fries',
               49: u'italian'}

In [106]:
all_numbers = list(range(0,50))

df_topics = pd.DataFrame(columns =["topic_name"])

for topic_number in all_numbers:
    df_topics = df_topics.append({
     "topic_name": topic_names[topic_number]
      }, ignore_index=True)
#print(df_topics)

In [124]:
clmns = df_topics['topic_name'].tolist()

In [126]:
#clmns

In [127]:
all_cols =  clmns
all_cols.insert(0,'business_id')
#print(all_cols)

In [132]:
clmns = df_topics['topic_name'].tolist()

In [128]:
lda_read.head()

,business_id,chinese,thai,healthy,smell,japanese,toronto,service,experience,asian soup,...,deli,buffet,bbq,french,money,street taco,pizza,airport & delivery,burger & fries,italian
0,DZWAS8k2F18CrAojI8meuQ,0.0000,0.0000,0.0000,0.0623,0.0,0.0,0.0000,0.0000,0.0,...,0.0000,0.000,0.0,0.0171,0.0276,0.0000,0.0593,0.0886,0.0,0.1217
1,0i95sgY7pzYW9k88SOVJ8A,0.2733,0.1195,0.1067,0.0000,0.0,0.0,0.0000,0.0000,0.0,...,0.0000,0.044,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.0000
2,MjBp8R5yOPdcQ_-5mpNIpg,0.0000,0.0221,0.1126,0.0000,0.0,0.0,0.0000,0.0000,0.0,...,0.0000,0.000,0.0,0.0000,0.0214,0.0000,0.0000,0.0279,0.0,0.0601
3,nGqj3Hjx_daTMzFke7g6aQ,0.0000,0.0140,0.0000,0.0000,0.0,0.0,0.0412,0.0468,0.0,...,0.0000,0.000,0.0,0.0000,0.0281,0.1107,0.0000,0.0156,0.0,0.0000
4,QfD_KzfBGRZ1OLu1Vs1rrA,0.0000,0.1261,0.0000,0.0000,0.0,0.0,0.0000,0.0000,0.0,...,0.2102,0.000,0.0,0.0000,0.0138,0.0000,0.0165,0.0000,0.0,0.1548


In [133]:
lda_read.columns = all_cols

In [130]:
lda_read

,business_id,chinese,thai,healthy,smell,japanese,toronto,service,experience,asian soup,...,deli,buffet,bbq,french,money,street taco,pizza,airport & delivery,burger & fries,italian
0,DZWAS8k2F18CrAojI8meuQ,0.0000,0.0000,0.0000,0.0623,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0171,0.0276,0.0000,0.0593,0.0886,0.0000,0.1217
1,0i95sgY7pzYW9k88SOVJ8A,0.2733,0.1195,0.1067,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0440,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
2,MjBp8R5yOPdcQ_-5mpNIpg,0.0000,0.0221,0.1126,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0214,0.0000,0.0000,0.0279,0.0000,0.0601
3,nGqj3Hjx_daTMzFke7g6aQ,0.0000,0.0140,0.0000,0.0000,0.0000,0.0000,0.0412,0.0468,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0281,0.1107,0.0000,0.0156,0.0000,0.0000
4,QfD_KzfBGRZ1OLu1Vs1rrA,0.0000,0.1261,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.2102,0.0000,0.0000,0.0000,0.0138,0.0000,0.0165,0.0000,0.0000,0.1548
5,dAmVTQ6ukuLh4UxTmBoRkg,0.0000,0.0254,0.0000,0.0000,0.0000,0.0000,0.0000,0.1337,0.0000,...,0.0126,0.0000,0.0000,0.0326,0.0266,0.0000,0.0000,0.0167,0.0000,0.0329
6,62HZvu4RyAnhRswWn3e7qA,0.0273,0.0104,0.0159,0.0000,0.0000,0.0000,0.2893,0.0156,0.0000,...,0.0242,0.0000,0.0000,0.0000,0.0138,0.0000,0.0000,0.0000,0.0142,0.0401
7,sLGQo2X21aOPSf6lKjeUTw,0.2510,0.2960,0.0000,0.0000,0.0000,0.0000,0.1908,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
8,oRRU93M-gYqcnes16GuZTw,0.0441,0.2584,0.0156,0.0000,0.0000,0.0000,0.0424,0.0000,0.0000,...,0.0000,0.0000,0.0194,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0356
9,2bYIP52sBrmZoVzziH2kfw,0.0000,0.0674,0.0000,0.0000,0.0574,0.0000,0.0000,0.0959,0.0000,...,0.0000,0.0808,0.0243,0.0000,0.0577,0.0000,0.0000,0.0000,0.0000,0.0000


In [134]:
lda_read[clmns]

,chinese,thai,healthy,smell,japanese,toronto,service,experience,asian soup,grocery,...,deli,buffet,bbq,french,money,street taco,pizza,airport & delivery,burger & fries,italian
0,0.0000,0.0000,0.0000,0.0623,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0171,0.0276,0.0000,0.0593,0.0886,0.0000,0.1217
1,0.2733,0.1195,0.1067,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0461,...,0.0000,0.0440,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
2,0.0000,0.0221,0.1126,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0190,...,0.0000,0.0000,0.0000,0.0000,0.0214,0.0000,0.0000,0.0279,0.0000,0.0601
3,0.0000,0.0140,0.0000,0.0000,0.0000,0.0000,0.0412,0.0468,0.0000,0.0116,...,0.0000,0.0000,0.0000,0.0000,0.0281,0.1107,0.0000,0.0156,0.0000,0.0000
4,0.0000,0.1261,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0668,...,0.2102,0.0000,0.0000,0.0000,0.0138,0.0000,0.0165,0.0000,0.0000,0.1548
5,0.0000,0.0254,0.0000,0.0000,0.0000,0.0000,0.0000,0.1337,0.0000,0.0000,...,0.0126,0.0000,0.0000,0.0326,0.0266,0.0000,0.0000,0.0167,0.0000,0.0329
6,0.0273,0.0104,0.0159,0.0000,0.0000,0.0000,0.2893,0.0156,0.0000,0.0103,...,0.0242,0.0000,0.0000,0.0000,0.0138,0.0000,0.0000,0.0000,0.0142,0.0401
7,0.2510,0.2960,0.0000,0.0000,0.0000,0.0000,0.1908,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
8,0.0441,0.2584,0.0156,0.0000,0.0000,0.0000,0.0424,0.0000,0.0000,0.0431,...,0.0000,0.0000,0.0194,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0356
9,0.0000,0.0674,0.0000,0.0000,0.0574,0.0000,0.0000,0.0959,0.0000,0.0572,...,0.0000,0.0808,0.0243,0.0000,0.0577,0.0000,0.0000,0.0000,0.0000,0.0000


In [135]:
lda_std = stats.zscore(lda_read[clmns])

In [136]:
#Cluster with kmeans
kmeans = KMeans(n_clusters=5, random_state=0).fit(lda_std)
labels = kmeans.labels_

In [137]:
#Glue back to originaal data
lda_read['clusters'] = labels

In [138]:
#Add the column into our list
clmns.extend(['clusters'])

In [139]:
#Lets analyze the clusters
print(lda_read[clmns].groupby(['clusters']).mean())

           chinese      thai   healthy     smell  japanese   toronto  \
clusters                                                               
0         0.008356  0.007978  0.022937  0.004811  0.003115  0.003957   
1         0.007234  0.006126  0.006343  0.003073  0.004535  0.002908   
2         0.005641  0.010331  0.010530  0.003928  0.001985  0.003870   
3         0.140395  0.100785  0.014770  0.002509  0.048658  0.003876   
4         0.002197  0.002448  0.003448  0.000607  0.001678  0.003214   

           service  experience  asian soup   grocery    ...         deli  \
clusters                                                ...                
0         0.030756    0.020018    0.002788  0.046288    ...     0.047771   
1         0.091377    0.043669    0.000845  0.008534    ...     0.006559   
2         0.050610    0.016756    0.001100  0.019755    ...     0.015142   
3         0.065552    0.023969    0.003021  0.013753    ...     0.002779   
4         0.003848    0.001901    0.002

In [140]:
lda_read.columns

Index(['business_id', 'chinese', 'thai', 'healthy', 'smell', 'japanese',
       'toronto', 'service', 'experience', 'asian soup', 'grocery', 'parking',
       'bar ambiance', 'uk', 'good service', 'fun ambiance', 'young',
       'comfort food', 'greek', 'high end', 'hotwing', 'breakfast', 'sweet',
       'wine & dine', 'pubs', 'good taste', 'na drinks', 'desserts',
       'coffee shop', 'mexican', 'reviews', 'new york', 'general restaurant',
       'beach', 'location', 'happy hour', 'amazing', 'vietnamese', 'time',
       'vas legas', 'montreal', 'deli', 'buffet', 'bbq', 'french', 'money',
       'street taco', 'pizza', 'airport & delivery', 'burger & fries',
       'italian', 'clusters'],
      dtype='object')